In [1]:
import numpy as np
import pandas as pd
import glob
import time

In [2]:
DataPath1 = "I:/DataSets/CIBR_data/FL/*"
DataPath2 = "I:/DataSets/CIBR_data/insight/*"
savePath = "I:/DataSets/CIBR_data/Slices_Norm_FL/"
fs = 500
duration = 30 * fs
sleep_dict = {"W":0, "N1": 0, "N2": 1, "N3": 2, "R": 3}

In [3]:
channel = np.load('I:/DataSets/CIBR_data/FL\\003/003_eeg_channel.npy')
print(channel)
# Pick Channel: 2, 6, 7
print("Pick Channel:", channel[2], channel[6], channel[7])

['F3-M2' 'F4-M1' 'C3-M2' 'C4-M1' 'O1-M2' 'O2-M1' 'REOG' 'LEMG' 'REMG'
 'LEOG']
Pick Channel: C3-M2 REOG LEMG


In [36]:
subjects_FL = glob.glob(DataPath1)
FL_DataPath = []
FL_Label = []
index = []

In [37]:
for name in subjects_FL:
    indx = name[-3:]
    index.append(indx)
    FL_DataPath.append(name + "/%s_eeg_data.npy" % indx)
    FL_Label.append(name + "/hypno_30s.csv")

In [51]:
for name in range(len(FL_DataPath)):
    time0 = time.time()
    label_total = pd.read_csv(FL_Label[name])
    onsets = list(label_total["onset"])
    for i in range(len(onsets)):
        onsets[i] = int(onsets[i] * fs)
    description = list(label_total["description"])
    for i in range(len(description)):
        description[i] = sleep_dict[description[i]]

    data = np.load(FL_DataPath[name])
    shape = data.shape
    count = len(description)
    print(count)
    
    data_C3 = data[2]
    data_REOG = data[6]
    data_EMG = data[7]
    data = np.array([data_C3, data_REOG, data_EMG])

    for i in range(count):
        data_temp = data[:, onsets[i]:onsets[i] + duration]
        saveName = savePath + "%s-%04d-stage%d.npy" % (index[name], i+1, description[i])
        np.save(saveName, data_temp)
    
    print("Done! Use Time: %.3f" % (time.time() - time0))
        

809
747


OSError: problem writing element 512 to file